# Solar energy forecast
---

### Import libraries

Import libraries for data cleaning, statistics, machine learning and visualization

In [ ]:
using Dates, DataFrames, Plots, StatsPlots, Statistics, Distributions, ConformalPrediction
using StatsBase, HypothesisTests, LinearAlgebra, Random, MLJ, CSV, CategoricalArrays 

### Dataset 

Our data contains information on these key factors:

+ **timedate**: date and hour of each datapoint
+ **WindSpeed**: wind speed in Km/h 
+ **Sunshine**: minutes per hours that sun is not cover by clouds (scaled 0-60)
+ **AirPressure**: athmosphere pressure in hPa
+ **Radiation**: solar radiation W/m^2^
+ **AirTemperature**: air temperature in Celsius
+ **RelativeAirHumidity**: relative humidity (scaled 0-100)
+ **SystemProduction**: system production kWh 

In [ ]:
DT = CSV.read("C:/Users/nicol/Documents/solar_1/Solar Power Plant Data.csv", DataFrame);
show(describe(DT),allcols = true)

In the table above we don't have any real `missing` data but we can see some problematic values (like the negative values for solar radiation). We assume that each negative value of solar radiation is due to some data trasmission error so we set all the negative value equal to zero, in addition we parse the data related to the hour and date to `timedate` format

In [ ]:
DT[DT.Radiation .< 0,:Radiation] .= 0.0;

DT.day = parse.(Int64,chop.(DT[:,"Date-Hour(NMT)"], head = 0, tail = 14))
DT.month = parse.(Int64,chop.(DT[:,"Date-Hour(NMT)"], head = 3, tail = 11))
DT.hour = parse.(Int64,chop.(DT[:,"Date-Hour(NMT)"], head = 11, tail = 3));

rename!(DT,"Date-Hour(NMT)" => "timedate");

# Create column with right formatting
DT.timedate_real = DateTime.(2017,DT.month,DT.day,DT.hour);
DT.date_real = Date.(2017,DT.month,DT.day);

Let's visualize the `SystemProduction` for each hour: 

In [ ]:
plot(DT.timedate_real, DT.SystemProduction, 
    title="Hourly production", label= :none, size=(750,300))

From the plot we notice that we have some multiple consecutive days where the production is zero (for example in Jan, May, Dec). It seems that instead of `missing` value we have some `zero` value when we don't have available data. We need to be very careful while performing the data cleaning because during the night the actual production of the solar panel is zero. Let's group by day and check the days with zero production  

In [ ]:
df = groupby(DT, :date_real)
dt = combine(df, 
             ["SystemProduction","WindSpeed",
             "Sunshine","AirPressure",
              "Radiation","AirTemperature",
              "RelativeAirHumidity","month"] .=> [sum, mean, mean, 
                                                            mean, mean, mean, 
                                                            mean, mean]; 
    renamecols = true);
sort!(dt,:date_real);

p1 = scatter(dt.Radiation_mean, dt.SystemProduction_sum, title = "Production vs Radiation (day)", 
             label= :none)
p1 = vline!([40], label= :none)
p1 = hline!([1800], label= :none)
p2 = scatter(DT.Radiation, DT.SystemProduction, title = "Production vs Radiation (hour)", 
             label= :none)

plot(p1, p2, layout=(1,2), size=(750,300))

From the plot is clear that we have some outlier where radiation is grater than 40 W/m^2^ and production is lower than 1800 kWh per day. Let's mark those days as *suspicious* and check if we have hour that can have some suspiciuos data, then compute the correlation between variables in the clean dataset

In [ ]:
suspect_day = dt[(dt.Radiation_mean .> 40) .&& (dt.SystemProduction_sum .< 1800),:date_real]
filter!([:date_real, :SystemProduction, :Radiation] => (x,y,z) -> x ∉ Ref(suspect_day) && 
        !(y == 0 && z > 40), DT)
cor(Matrix(DT[:,2:8]))

Last step before the machine learning model is to include the time into a numerical variable using **Cyclical Encoder**. This methods allow us to take into account the time cyclicity for months, days, hours

In [ ]:
function cyclical_encoder(df::DataFrame, columns::Union{Array, Symbol}, max_val::Union{Array, Int} )
    for (column, max) in zip(columns, max_val)        
        df[:, Symbol(string(column) * "_sin")] = sin.(2*pi*df[:, column]/max)
        df[:, Symbol(string(column) * "_cos")] = cos.(2*pi*df[:, column]/max)
    end
    return df
end

cyclical_encoder(DT, ["day","month","hour"], [31,12,23]);

## Machine Learning

EvoTrees is a regression algorithm in Julia library for creating gradient boosting regression models. It allows you to build decision trees efficiently, focusing on performance. EvoTrees works by combining multiple weaker decision trees into a stronger final model. It supports various loss functions specifically designed for regression tasks, which guide the training process and evaluate how well your model performs. The library utilizes histogram-based algorithms for faster data processing and can also handle different types of features within your data, including categorical ones. Overall, EvoTrees provides a versatile toolkit for building regression models in Julia using gradient boosting. 

Split the dataset in train and test considering only the hours with solar radiation grater than zero (exclude nights and evenings). We consider train from 01/01 to 06/30 and test from 07/01 to 12/31. We perform hourly estimation and then we group by day of the year. We include all the available variable into the model: 

In [ ]:
DT_model = DT[DT.Radiation .> 0.0,:]
train, test = (collect(1:1828),collect(1829:3744));
X = DT_model[:,vcat(2:7,14:19)];
y = DT_model[:,:SystemProduction];

We need to load the `EvoTreeRegressor` algorithm, set the parameters, create the machine and cross validate the model using 5-folds repeating the operation for 5 times

In [ ]:
EvoTreeRegressor = MLJ.@load EvoTreeRegressor pkg=EvoTrees;
et_regressor = EvoTreeRegressor(nbins = 32, max_depth = 10, nrounds = 200);

model_glm = et_regressor;
mach_glm = machine(model_glm, X, y); 
fit!(mach_glm, rows = train);

# Cross-validation
evaluate!(mach_glm, resampling = CV(nfolds=5, rng=1234), 
          repeats=5, measure = [rmse, rsquared], verbosity=0);

### Results

In [ ]:
DT_model.predicts = zeros(size(DT_model,1))        
DT_model[test,:predicts] .= MLJ.predict(mach_glm, rows=test)
histogram(MLJ.predict(mach_glm, rows=test))

df = groupby(DT_model, :date_real)
dt = combine(df, ["SystemProduction","predicts"] .=> [sum, sum]; renamecols = true);
sort!(dt,:date_real);
dt = dt[dt.date_real .> Date.(2017,6,30),:]

q2 = plot(dt[:,:date_real],dt[:,:SystemProduction_sum],  title = "Actual vs Predict", label = "Actual")
q2 = plot!(dt[:,:date_real],dt[:,:predicts_sum], mc = :orange, label = "Predict")
q1 = scatter(dt[:,:SystemProduction_sum],dt[:,:predicts_sum], title = "Actual vs Predict", label = :none)
q1 = plot!(collect(0:59000),collect(0:59000), label = :none, mc = :red)


plot(q1, q2, layout=(1,2), size=(750,300))

In [ ]:
println("RMSE: ", string.(rmse(dt[:,:SystemProduction_sum],dt[:,:predicts_sum])))
println("MAE: ", string.(mae(dt[:,:SystemProduction_sum],dt[:,:predicts_sum])))
println("R²: ", string.(cor(dt[:,:SystemProduction_sum],dt[:,:predicts_sum]).^2))